In [1]:
import pandas as pd

In [ ]:
import xml.etree.ElementTree as ET
file_name = "data/remybd_Space_Invaders.kml"
tree = ET.parse(file_name)
root = tree.getroot()
docu = root[0]
# placemark_list = get_all_placemarks(calc_list)


initial_size : 1310
sum : 1310


In [10]:
placemark_list

[<Element '{http://www.opengis.net/kml/2.2}Placemark' at 0x000002A13DE6F8D0>,
 <Element '{http://www.opengis.net/kml/2.2}Placemark' at 0x000002A13DE6EF70>,
 <Element '{http://www.opengis.net/kml/2.2}Placemark' at 0x000002A13DE6EED0>,
 <Element '{http://www.opengis.net/kml/2.2}Placemark' at 0x000002A13DE6E430>,
 <Element '{http://www.opengis.net/kml/2.2}Placemark' at 0x000002A13E48F1A0>,
 <Element '{http://www.opengis.net/kml/2.2}Placemark' at 0x000002A13E45C400>,
 <Element '{http://www.opengis.net/kml/2.2}Placemark' at 0x000002A13E45CEA0>,
 <Element '{http://www.opengis.net/kml/2.2}Placemark' at 0x000002A13E45C630>,
 <Element '{http://www.opengis.net/kml/2.2}Placemark' at 0x000002A13E946020>,
 <Element '{http://www.opengis.net/kml/2.2}Placemark' at 0x000002A13E946390>,
 <Element '{http://www.opengis.net/kml/2.2}Placemark' at 0x000002A13E946700>,
 <Element '{http://www.opengis.net/kml/2.2}Placemark' at 0x000002A13E946A70>,
 <Element '{http://www.opengis.net/kml/2.2}Placemark' at 0x00000

In [33]:
import pandas as pd
import xml.etree.ElementTree as ET
import re
# Define a function to parse the KML file and extract names and coordinates.
def parse_kml_to_dataframe(file_path):
    """
    Parse a KML file to extract name, latitude, and longitude of each placemark
    and return the data as a pandas DataFrame.

    :param file_path: Path to the KML file
    :return: A pandas DataFrame with columns 'Name', 'Latitude', 'Longitude'
    """
    placemarks = []

    # Parse the KML file
    tree = ET.parse(file_path)
    root = tree.getroot()

    # Define the KML namespace
    namespaces = {'kml': 'http://www.opengis.net/kml/2.2'}

    # Iterate over all Placemark elements
    for placemark in root.findall(".//kml:Placemark", namespaces):
        # Extract name
        name = placemark.find("kml:name", namespaces)
        name_text = name.text if name is not None else "Unknown"

        # Extract coordinates and split into longitude and latitude
        coordinates = placemark.find(".//kml:coordinates", namespaces)
        if coordinates is not None:
            coords = coordinates.text.strip().split(",")
            longitude, latitude = coords[0], coords[1]
        else:
            longitude, latitude = "0", "0"  # Default to 0 if coordinates are missing

        # Append to the list as a dictionary
        placemarks.append({
            'Name': name_text,
            'Latitude': float(latitude),
            'Longitude': float(longitude)
        })

    # Convert to a pandas DataFrame
    return pd.DataFrame(placemarks)
def standardize_code(code):
    # Handle cases where the input is not a string
    if not isinstance(code, str):
        return code
    # Define a function to standardize the string format
    match = re.match(r"PA.{1}(\d{2,4})", code)
    if match:
        # Extract the number part and pad with zeros to make it 4 digits
        number_part = match.group(1)
        standardized_code = f"PA_{int(number_part):04d}"
        return standardized_code
    return code  # Return the original code if it doesn't match the pattern


In [44]:
file_name = "data/remybd_Space_Invaders.kml"
placemark_df = parse_kml_to_dataframe(file_name)
placemark_df['code'] = placemark_df['Name'].apply(lambda x: re.findall(r'PA.{1}[0-9]{2,4}', x))
placemark_df = placemark_df.explode('code')
placemark_df['code'] = placemark_df['code'].str.replace("-","_")
placemark_df['code'] = placemark_df['code'].apply(standardize_code)
placemark_df['localisation'] = placemark_df['Name'].str.replace(pat=r'PA.{1}[0-9]{2,4}',repl="",regex=True)
placemark_df['localisation'] = placemark_df['localisation'].str.replace(pat=r"^[ ^:&-]*",repl="",regex=True)
placemark_df[['Latitude','Longitude','code','localisation']].to_excel("data/remybd_clean_data.xlsx")

In [ ]:
placemark_df

,Name,Latitude,Longitude,code
0,PA_1200: 37 Rue de la Bûcherie,48.852537,2.347295,PA_1200
1,PA_1201: 14 Rue Ternaux,48.864395,2.373227,PA_1201
2,PA_1202: 73 Rue de Prony,48.883344,2.301604,PA_1202
3,PA_1203: 64 Rue Monsieur le Prince,48.847838,2.340887,PA_1203
4,PA_1204: 65 rue d'Avron\n,48.852327,2.403291,PA_1204
...,...,...,...,...
1306,PA-543 : Duchefdelaville - Chevaleret,48.832648,2.370933,PA_543
1307,PA-057 & PA-460: 38 Louise Weiss,48.834396,2.369657,PA_057
1307,PA-057 & PA-460: 38 Louise Weiss,48.834396,2.369657,PA_460
1308,PA-459 : Métro Chevaleret,48.834920,2.368353,PA_459
